In [1]:
!pip install transformers datasets sentencepiece accelerate evaluate 

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.2 MB ? eta -:--:--
    --------------------------------------- 0.5/26.2 MB 932.9 kB/s eta 0:00:28
   - -------------------------------------- 1.0/26.2 MB 1.5 MB/s eta 0:00:17
   - -------------------------------------- 1.3/26.2 MB 1.5 MB/s eta 0:00:17
   ----- ---------------------------------- 3.4/26.2 MB 3.1 MB/s eta 0:00:08
   -------- ------------------------------- 5.8/26.2 MB 4.4 MB/s eta 0:00:05
   ----------- ---------------------------- 7.9/26.2 MB 5.2 MB/s eta 0:00:04
   --------------- ------------------------ 10.2/26.2 MB 5.9 MB/s eta 0:00:03
   ------------------ --------------------- 12.3/26.2 MB 6.3 MB/s eta 0:00:03
   --------------------- ------------------ 14.4/26.2 MB 6.8 MB/s eta 0:00:02
   -------------

In [2]:
!pip install torch

In [3]:
import torch , transformers

c:\Users\Asif Computer\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [7]:
print('version of torch:', torch.__version__)
print ('version of transformer:',transformers.__version__)
print ('check cuda is available :', torch.cuda.is_available())

version of torch: 2.7.0+cpu
version of transformer: 4.48.3
check cuda is available : False


In [8]:
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM
model_name = "t5-small" 

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


c:\Users\Asif Computer\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asif Computer\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [15]:
examples = ["translate english into german: this house is wonderful.",
            "summarize: the quick brown fox jumpsover the lazy dog. the dog was not amused "]
for text in examples:
    input = tokenizer(text , return_tensors="pt", truncation=True)
    output = model.generate(**input, max_length = 64, num_beams=4, early_stopping=True)
    print("input:", text)
    print("output", tokenizer.decode(output[0],skip_special_tokens=True))

input: translate english into german: this house is wonderful.
output Dieses Haus ist wunderbar.
input: summarize: the quick brown fox jumpsover the lazy dog. the dog was not amused 
output the quick brown fox jumpsover the lazy dog was not amused.


In [17]:
notes = [ "convert it into german: my name is bilal and i am new to to this software house ",
         "summarize : today was my statistics paper the paper was quite difficult  but  i was prepared for it and that helps me alot "]
for note in notes:
    inputs = tokenizer(note, return_tensors="pt",truncation=True)
    outputs = model.generate(**inputs,max_length=64,num_beams=4, early_stopping=True)

    print("outputs:",tokenizer.decode(outputs[0],skip_special_tokens=True))

outputs: convert it into german: my name is bilal and i am new to this software house
outputs: today was my statistics paper the paper was quite difficult but i was prepared for it and that helps me alot.


In [19]:
s = "the quick brown fox jumps over the lazy dog."
tokens = tokenizer.tokenize(s)
ids = tokenizer(s).input_ids
print (tokens)

['▁the', '▁quick', '▁brown', '▁', 'fox', '▁jump', 's', '▁over', '▁the', '▁lazy', '▁dog', '.']


In [20]:
print ("decoded back :", tokenizer.decode(ids))

decoded back : the quick brown fox jumps over the lazy dog.</s>


In [29]:
prompt = "parapharase : Artifiv=cial intelligence is channging the world"
params = [
    {"num_beams":1,"do_sample":True, "temperature":1.0},
    {"num_beams":2,"do_sample":False},
    {"num_beams":4,"do_sample":True, "top_p":0.9,"temperature":0.8}
]
for p in params:
    input=tokenizer(prompt,return_tensors="pt")
    out = model.generate(**inputs, max_new_tokens=40, **p)
    print(p, "=>", tokenizer.decode(out[0], skip_special_tokens=True))

{'num_beams': 1, 'do_sample': True, 'temperature': 1.0} => today was my statistics paper, the paper was quite difficult but i was prepared for it and that helps me alot.
{'num_beams': 2, 'do_sample': False} => the paper was quite difficult but i was prepared for it and that helps me alot.
{'num_beams': 4, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.8} => today was my statistics paper the paper was quite difficult but i was prepared for it and that helps me alot.
